In [1]:
import os
import sys
import platform

import torch
import torch.nn
import torchvision.datasets


from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.utils import save_image

In [2]:
print("Python Version: {}".format(platform.python_version()))
print("Torch Version: {}".format(torch.__version__))

Python Version: 3.7.1
Torch Version: 0.4.1


# Get MNIST Data

In [3]:
num_epochs = 11
batch_size = 128
learning_rate = 1e-3

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))])

In [5]:
mnistTrainData = torchvision.datasets.MNIST("Data", download=True, train=True, transform=transform)
mnistTestData = torchvision.datasets.MNIST("Data", download=True, train=False, transform=transform)

In [6]:
print("Number of Training Samples: {}".format(len(mnistTrainData)))
print("Number of Testing Samples: {}".format(len(mnistTestData)))

Number of Training Samples: 60000
Number of Testing Samples: 10000


In [7]:
trainDataLoader = torch.utils.data.DataLoader(mnistTrainData, batch_size=batch_size, shuffle=True)
testDataLoader = torch.utils.data.DataLoader(mnistTestData, batch_size=batch_size, shuffle=True)

# Autoencoder Model

In [8]:
class Autoencoder(torch.nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = torch.nn.Sequential(torch.nn.Linear(28 * 28, 64))
        self.decoder = torch.nn.Sequential(torch.nn.Linear(64, 28 * 28), torch.nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Train Model

In [9]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [10]:
model = Autoencoder()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(trainDataLoader):
        img = data.view(data.size(0), -1)
        img = Variable(img)
        
        output = model(img)
        loss = criterion(output, img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.data.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, 'image_{}.png'.format(epoch))


epoch [1/11], loss:0.0469
epoch [2/11], loss:0.0305
epoch [3/11], loss:0.0254
epoch [4/11], loss:0.0214
epoch [5/11], loss:0.0184
epoch [6/11], loss:0.0168
epoch [7/11], loss:0.0181
epoch [8/11], loss:0.0170
epoch [9/11], loss:0.0174
epoch [10/11], loss:0.0138
epoch [11/11], loss:0.0143


# Visualize Ouput

# Visualize Latent Properties